In [1]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import tqdm as tqdm
from lxml import html
import time

In [3]:
def getSoup_test(link):
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'html')

In [23]:
bf_test = getSoup_test('https://www.bezrealitky.cz/nemovitosti-byty-domy/595733-nabidka-prodej-bytu')

In [33]:
bf_table = bf_test.find('div',{'data-element':"detail-description"}).find('table')

In [92]:
table_extract = [i.text.strip() for i in bf_table.findAll('tr')]
table_extract

['Internet:\n\nZjistěte rychlost O2 internetu u vás doma.',
 'Číslo inzerátu:\n595733',
 'Dispozice:\n3+kk',
 'Plocha:\n96 m²',
 'Cena:\n7.590.000 Kč\n\nChci hypotéku\n\ni',
 'Město:\nPraha',
 'Městská část:\nZličín',
 'Typ vlastnictví:\nOsobní',
 'Typ budovy:\nCihla',
 'Vybavenost:\nVybavený\nChci vybavit s Bonami',
 'Podlaží:\n5',
 'Balkón:\nAno',
 'Terasa:\nNe',
 'Výtah:\nNe',
 'Garáž:\nNe',
 'K dispozici od:\nIhned']

In [72]:
data_frame = pd.DataFrame(table_extract)

In [199]:
data_frame_recon = data_frame[0].str.split("\n", expand = True)
df = pd.DataFrame(data_frame_recon[[0,1]])
df.columns = ['metric', 'value']
df = df.set_index('metric')
df

,value
metric,
Internet:,
Číslo inzerátu:,595733
Dispozice:,3+kk
Plocha:,96 m²
Cena:,7.590.000 Kč
Město:,Praha
Městská část:,Zličín
Typ vlastnictví:,Osobní
Typ budovy:,Cihla


In [149]:
bf_map = bf_test.find('div',{'id':"map"})
x = bf_map.find('iframe')['src'].find('q=') + 2
y = bf_map.find('iframe')['src'].find('&key')
location = bf_map.find('iframe')['src'][x:y]
print(location)
str_test = location.split(',')
str_test[0]

50.0568199,14.2850438


'50.0568199'

In [200]:
loc_dict = {"lat" : [str_test[0]], "long": [str_test[1]]}
loc_dict
loc_df = pd.DataFrame.from_dict(loc_dict, orient = 'index', columns = ['value'])
df_upd = pd.concat([df, loc_df])
df_upd

,value
Internet:,
Číslo inzerátu:,595733
Dispozice:,3+kk
Plocha:,96 m²
Cena:,7.590.000 Kč
Město:,Praha
Městská část:,Zličín
Typ vlastnictví:,Osobní
Typ budovy:,Cihla
Vybavenost:,Vybavený


In [25]:
class HouseForSale: 
    '''
    Getting infos from webpage 
    '''
    def __init__(self, link):
        '''
        Input link to search Books appered in 
        '''
        self.link = link

    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'html')
    
    def getParameters(self):
        bf = self.getSoup()
        bf_table = bf.find('div',{'data-element':"detail-description"}).find('table')
        table_extract = [i.text.strip() for i in bf_table.findAll('tr')]
        df_table = pd.DataFrame(table_extract)
        df_table_split = df_table[0].str.split("\n", expand = True)
        df = pd.DataFrame(df_table_split[[0,1]])
        df.columns = ['metric', 'value']
        df = df.set_index('metric')
        return df
    
    def getMap(self):
        bf = self.getSoup()
        bf_map = bf.find('div',{'id':"map"})
        x = bf_map.find('iframe')['src'].find('q=') + 2
        y = bf_map.find('iframe')['src'].find('&key')
        location = bf_map.find('iframe')['src'][x:y]
        loc_str = location.split(',')
        loc_dict = {"lat" : [loc_str[0]], "long": [loc_str[1]]}
        loc_df = pd.DataFrame.from_dict(loc_dict, orient = 'index', columns = ['value'])
        return loc_df
    
    def getDf(self):
        df_par = self.getParameters()
        df_map = self.getMap()
        df = pd.concat([df_par, df_map])
        return df

In [26]:
HouseForSale('https://www.bezrealitky.cz/nemovitosti-byty-domy/595733-nabidka-prodej-bytu').getDf()

,value
Internet:,
Číslo inzerátu:,595733
Dispozice:,3+kk
Plocha:,96 m²
Cena:,7.590.000 Kč
Město:,Praha
Městská část:,Zličín
Typ vlastnictví:,Osobní
Typ budovy:,Cihla
Vybavenost:,Vybavený


In [14]:
class Downloader:
    '''
    Download all links for bookchecker to initiate
    '''
    def __init__(self, link):
        '''
        Give link to extract links of given webpages
        '''
        self.link = link
    
    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'lxml')

    def getLinks(self):
        wp = self.getSoup()
        links = wp.find('span',{'id':'Unseen_University_and_the_Wizards'}).parent.previous.previous.parent.parent.parent.find_all('li')
        return ['http://wiki.lspace.org' + li.find('a')['href'] for li in links]

    def getDataFrame(self):
        links = self.getLinks()
        name_list = [BookTaker(i).getName() for i in links]
        books_list = [BookTaker(i).getBooks() for i in links]
        return pd.DataFrame({"Character":name_list, "Books":books_list})
        